# Part V: Utilizing AI

## Beeing Specific Matters!
The following example shows, why it is important to be specific and know what you want, because AI may not by default provide the best result.
(Howver, if you ask for the most performant implementation, it will probably work as well. The most important thing is to be specific!)

### 1st Promt
"How can I sum the squares of numbers from 1 to N in Python?"

In [ ]:
%%time
N = 10000
for j in range(100):
    result = sum(i**2 for i in range(1, N + 1))

### 2nd Promt
"How can I sum the squares of numbers from 1 to N in python with numpy"

In [ ]:
%%time
import numpy as np
N = 10000
for j in range(100):
    result = np.sum(np.arange(1, N + 1)**2)

In [ ]:
%%time
def sum_squares_py(N: int) -> int:
    return sum(i * i for i in range(1, N + 1))

N = 10000
for j in range(100):
    sum_squares_py(N)

### 3rd Prompt
"How can I sum the squares of numbers from 1 to N in Python? Please provide the most performant implementation, except the close formular"

In [ ]:
%%time
def sum_squares_numpy(N: int) -> int:
    arr = np.arange(1, N + 1, dtype=np.int64)
    return np.sum(arr * arr)

N = 10000
for j in range(100):
    sum_squares_py(N)

______________

## Plotting Excercise

**Exercise**:
```
Let’s generate some LHC mock data of a Z-> mu mu analysis and visualize it.

The data should contain:
pT mu1, pT mu2, raw pT mu1, raw pT mu2, corr1, corr2, theta mu1, theta mu2, phi mu1 phi mu2, pT Z, eta Z
Make sure that the distributions make sense (e.g. by visualizing them!)
+ Verify with some statistics checks
Now, please create plots:
1) The histogram of the Z-> mu mu candidate invariant mass, overlayed with mu1/2 distributions
2) Below, the pT Z distribution
3) Next line: polar (left)  and azimuth (right) distributions of both muons
4) In the next line, on the left, the mu1 pT distribution and the raw pT with a subplot of the average correction factor per bin below. The same on the right for mu2.
Optional: Use the mplhep package to make them look like CMS plots
Overlay ALL plots with “FAKE”
```

**Some Recommendations**:
- In case you are not memory bound, do the data loading once and keep the original data cached
- It is mostly always a good idea to start simple and extend it
- Even though it is very convenient to just c&p, at least try to shortly understand what AI recommends you to implement. Especially, make sure that the correct data is used, nothing is changed, and so on

### One Possible (AI) Solution

#### First: Generate the Mock Data

In [ ]:
# Final physics-consistent Z→μ⁺μ⁻ toy generator with calibrated raw/corrected pT
# - Generate Z kinematics: pT^Z (soft), eta_Z (central), phi_Z (uniform), m_Z (Gaussian around PDG)
# - Isotropic decay in Z rest frame, boost to lab
# - Apply small per-muon calibration factors to define raw/corrected pT
# - Drop any unphysical events

import numpy as np
import pandas as pd

# Constants
MZ_MEAN = 91.1876  # GeV
MZ_SIGMA = 2.5     # GeV (toy width; not a full detector resolution model)
MMU = 0.1056583745 # GeV

rng = np.random.default_rng(12345)
N = 10_000

def fourvec(E, px, py, pz):
    return np.stack([E, px, py, pz], axis=-1)

def boost(fv, beta):
    """Lorentz-boost four-vectors fv by 3-velocity beta (arrays broadcast)."""
    b2 = np.sum(beta**2, axis=-1, keepdims=True)
    # Ensure |beta|<1 (drop rare pathological numerics later if any)
    gamma = 1.0 / np.sqrt(1.0 - b2 + 1e-16)
    bp = np.sum(beta * fv[...,1:], axis=-1, keepdims=True)
    E_prime = gamma * (fv[...,0:1] + bp)
    factor = (gamma - 1.0) / (b2 + 1e-16)
    spatial = fv[...,1:] + factor * bp * beta + gamma * beta * fv[...,0:1]
    return np.concatenate([E_prime, spatial], axis=-1)

def phi_from_p(px, py):
    a = np.arctan2(py, px)
    a = np.where(a < 0, a + 2*np.pi, a)
    return a

def theta_from_p(px, py, pz):
    p = np.sqrt(px*px + py*py + pz*pz)
    cos_th = np.clip(pz / (p + 1e-16), -1.0, 1.0)
    return np.arccos(cos_th)

# --- 1) Generate Z bosons ---
# Soft pT spectrum (Gamma k=2, theta=12 GeV -> mean 24 GeV); tuneable for realism
pT_Z = rng.gamma(shape=2.0, scale=12.0, size=N)
phi_Z = rng.uniform(0, 2*np.pi, size=N)        # uniform
eta_Z = rng.normal(0.0, 1.0, size=N)           # central
m_Z = rng.normal(MZ_MEAN, MZ_SIGMA, size=N)    # toy mass width

px_Z = pT_Z * np.cos(phi_Z)
py_Z = pT_Z * np.sin(phi_Z)
pz_Z = pT_Z * np.sinh(eta_Z)
pZ_mag2 = px_Z**2 + py_Z**2 + pz_Z**2
E_Z = np.sqrt(m_Z**2 + pZ_mag2)

PZ = np.stack([px_Z, py_Z, pz_Z], axis=-1)
beta_Z = PZ / E_Z[:,None]

# --- 2) Z → μ⁺μ⁻ isotropic in Z rest frame ---
p_star = 0.5 * np.sqrt(np.maximum(m_Z**2 - 4*MMU**2, 0.0))   # two-body momentum
E_star = np.sqrt(p_star**2 + MMU**2)

cos_th = rng.uniform(-1.0, 1.0, size=N)  # uniform in cos(theta*)
sin_th = np.sqrt(np.maximum(1.0 - cos_th**2, 0.0))
phi_star = rng.uniform(0, 2*np.pi, size=N)

px1_star = p_star * sin_th * np.cos(phi_star)
py1_star = p_star * sin_th * np.sin(phi_star)
pz1_star = p_star * cos_th

mu1_rf = fourvec(E_star, px1_star, py1_star, pz1_star)
mu2_rf = fourvec(E_star, -px1_star, -py1_star, -pz1_star)

# --- 3) Boost to lab ---
mu1_lab = boost(mu1_rf, beta_Z)
mu2_lab = boost(mu2_rf, beta_Z)

E1, px1, py1, pz1 = mu1_lab[:,0], mu1_lab[:,1], mu1_lab[:,2], mu1_lab[:,3]
E2, px2, py2, pz2 = mu2_lab[:,0], mu2_lab[:,1], mu2_lab[:,2], mu2_lab[:,3]

# --- 4) Compute observables (lab) ---
pT_mu1_truth = np.sqrt(px1**2 + py1**2)
pT_mu2_truth = np.sqrt(px2**2 + py2**2)
phi_mu1 = phi_from_p(px1, py1)
phi_mu2 = phi_from_p(px2, py2)
theta_mu1 = theta_from_p(px1, py1, pz1)
theta_mu2 = theta_from_p(px2, py2, pz2)

# invariant mass of the pair (should peak near MZ_MEAN)
E_sum = E1 + E2
px_sum = px1 + px2
py_sum = py1 + py2
pz_sum = pz1 + pz2
m_mumu = np.sqrt(np.maximum(E_sum**2 - (px_sum**2 + py_sum**2 + pz_sum**2), 0.0))

# --- 5) Apply small per-muon calibration to define raw/corrected pT ---
corr_mu1 = rng.normal(1.01, 0.02, size=N).clip(0.95, 1.05)
corr_mu2 = rng.normal(1.00, 0.02, size=N).clip(0.95, 1.05)
pT_mu1_raw = pT_mu1_truth / corr_mu1
pT_mu2_raw = pT_mu2_truth / corr_mu2
pT_mu1_corr = pT_mu1_raw * corr_mu1
pT_mu2_corr = pT_mu2_raw * corr_mu2

# --- 6) Drop any unphysical results ---
mask = np.isfinite(E1) & np.isfinite(E2) & np.isfinite(pT_Z) & np.isfinite(eta_Z) & \
       np.isfinite(theta_mu1) & np.isfinite(theta_mu2) & np.isfinite(phi_mu1) & np.isfinite(phi_mu2) & \
       (E1 > 0) & (E2 > 0) & (m_mumu > 0)

# Optional gentle detector-like sanity: keep |eta_Z|<3.5 (broad), pT_Z<200 GeV to tame outliers
mask &= (np.abs(eta_Z) < 3.5) & (pT_Z < 200.0)

df = pd.DataFrame({
    "pT_mu1_raw": pT_mu1_raw[mask],
    "pT_mu2_raw": pT_mu2_raw[mask],
    "corr_mu1": corr_mu1[mask],
    "corr_mu2": corr_mu2[mask],
    "pT_mu1": pT_mu1_corr[mask],
    "pT_mu2": pT_mu2_corr[mask],
    "pT_Z": pT_Z[mask],
    "theta_mu1": theta_mu1[mask],
    "theta_mu2": theta_mu2[mask],
    "phi_mu1": phi_mu1[mask],
    "phi_mu2": phi_mu2[mask],
    "eta_Z": eta_Z[mask],
    "m_mumu": m_mumu[mask]
})

out_path = "muon_pairs_10000.csv"
df.to_csv(out_path, index=False)
out_path


#### Load the Data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("muon_pairs_10000.csv")

# Instect:
print(df.head())
print(df.describe())

#### Validate the Data

In [ ]:
print("⟨m_mumu⟩ ≈", df["m_mumu"].mean())
print("⟨pT_Z⟩   ≈", df["pT_Z"].mean())

fig, ax = plt.subplots(1,2, figsize=(10,4))
ax[0].hist(df["pT_Z"], bins=40)
ax[0].set_xlabel(r"$p_T^Z$ [GeV]"); ax[0].set_ylabel("Counts"); ax[0].set_title("Z $p_T$ (soft)")

ax[1].hist(df["m_mumu"], bins=80)
ax[1].set_xlabel(r"$m_{\mu\mu}$ [GeV]"); ax[1].set_ylabel("Counts"); ax[1].set_title("Z mass peak (~91 GeV)")
plt.tight_layout(); plt.show()


#### Plotting

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplhep as hep
from matplotlib.gridspec import GridSpec, GridSpecFromSubplotSpec

# --- CMS style setup ---
plt.style.use(hep.style.CMS)
#hep.cms.label("Preliminary", data=True, year=2022, lumi=138, loc=0)


# --- Derived proxies for illustration ---
m_mu1 = df["pT_mu1"] * np.cosh(df["eta_Z"] / 2)
m_mu2 = df["pT_mu2"] * np.cosh(df["eta_Z"] / 2)

# --- Binning ---
bins_mass  = np.linspace(df["m_mumu"].min(), df["m_mumu"].max(), 100)
bins_pT    = np.linspace(0, df["pT_Z"].max(), 40)
bins_theta = np.linspace(0, np.pi, 50)
bins_phi   = np.linspace(0, 2*np.pi, 50)
bins_raw   = np.linspace(df[["pT_mu1_raw","pT_mu2_raw"]].min().min(),
                         df[["pT_mu1_raw","pT_mu2_raw"]].max().max(), 30)

def avg_corr(raw, corr, bins):
    cats = pd.cut(raw, bins=bins, include_lowest=True)
    means = corr.groupby(cats).mean().to_numpy()
    centers = 0.5*(bins[1:]+bins[:-1])
    return centers, means

c_mu1_x, c_mu1_y = avg_corr(df["pT_mu1_raw"], df["corr_mu1"], bins_raw)
c_mu2_x, c_mu2_y = avg_corr(df["pT_mu2_raw"], df["corr_mu2"], bins_raw)

# --- Figure layout (wider figure) ---
fig = plt.figure(figsize=(16, 20))  # <-- wider figure
gs  = GridSpec(4, 2, height_ratios=[0.8, 0.8, 1, 1.3], figure=fig)

ax_mass = fig.add_subplot(gs[0, :])   # full-width Z mass
ax_pTZ  = fig.add_subplot(gs[1, :])   # full-width Z pT
ax_th   = fig.add_subplot(gs[2, 0])   # θ plots
ax_phi  = fig.add_subplot(gs[2, 1])   # φ plots

gsb_L   = GridSpecFromSubplotSpec(2, 1, subplot_spec=gs[3, 0], height_ratios=[3, 1])
gsb_R   = GridSpecFromSubplotSpec(2, 1, subplot_spec=gs[3, 1], height_ratios=[3, 1])
ax_mu1_hist = fig.add_subplot(gsb_L[0])
ax_mu1_corr = fig.add_subplot(gsb_L[1], sharex=ax_mu1_hist)
ax_mu2_hist = fig.add_subplot(gsb_R[0])
ax_mu2_corr = fig.add_subplot(gsb_R[1], sharex=ax_mu2_hist)

# --- Row 1: Z invariant mass (full range), with reference lines ---
counts_z, edges_z = np.histogram(df["m_mumu"], bins=bins_mass)
hep.histplot(counts_z, edges_z, ax=ax_mass, histtype="fill", label=r"$m_{\mu\mu}$", color="lightgray")

# annotate PDG M_Z and sample mean ± std
MZ_PDG = 91.1876
mu = df["m_mumu"].mean()
sigma = df["m_mumu"].std()

for x, lab, ls in [(MZ_PDG, r"$M_Z^{\rm PDG}$", "-."), (mu, r"$\langle m_{\mu\mu}\rangle$", "--")]:
    ax_mass.axvline(x, color="black", ls=ls, lw=1.2, label=lab)
ax_mass.axvspan(mu - sigma, mu + sigma, color="k", alpha=0.08, label=r"$\pm1\sigma$")

ax_mass.set_xlabel(r"$m_{\mu\mu}$ [GeV]")
ax_mass.set_ylabel("Events")
ax_mass.set_title(r"Z Candidate Invariant Mass (Full Range)")
ax_mass.legend(loc="upper right")

# --- Row 2: Z pT ---
counts_pT, edges_pT = np.histogram(df["pT_Z"], bins=bins_pT)
hep.histplot(counts_pT, edges_pT, ax=ax_pTZ, histtype="fill", color="lightgray")
ax_pTZ.set_xlabel(r"$p_T^Z$ [GeV]")
ax_pTZ.set_ylabel("Events")
ax_pTZ.set_title(r"Z Transverse Momentum Spectrum (Soft)")
ax_pTZ.legend().remove()

# --- Row 3: angular distributions ---
for col, lbl, c in zip(["theta_mu1","theta_mu2"], [r"$\theta_{\mu_1}$", r"$\theta_{\mu_2}$"], ["red","blue"]):
    vals, edges = np.histogram(df[col], bins=bins_theta)
    hep.histplot(vals, edges, ax=ax_th, histtype="step", label=lbl, color=c)
ax_th.set_xlabel(r"$\theta$ [rad]")
ax_th.set_ylabel("Events")
ax_th.set_title("Muon Polar Angle Distributions")
ax_th.legend(fontsize=11)

for col, lbl, c in zip(["phi_mu1","phi_mu2"], [r"$\phi_{\mu_1}$", r"$\phi_{\mu_2}$"], ["red","blue"]):
    vals, edges = np.histogram(df[col], bins=bins_phi)
    hep.histplot(vals, edges, ax=ax_phi, histtype="step", label=lbl, color=c)
ax_phi.set_xlabel(r"$\phi$ [rad]")
ax_phi.set_ylabel("Events")
ax_phi.set_title("Muon Azimuthal Angle Distributions")
ax_phi.legend(fontsize=11)

# --- Row 4: μ1 and μ2 pT + correction ---
for axH, axC, mu, color in [(ax_mu1_hist, ax_mu1_corr, "mu1", "red"), (ax_mu2_hist, ax_mu2_corr, "mu2", "blue")]:
    raw = df[f"pT_{mu}_raw"]
    cor = df[f"pT_{mu}"]
    vals_raw, edges_raw = np.histogram(raw, bins=bins_raw)
    vals_cor, _ = np.histogram(cor, bins=bins_raw)
    hep.histplot(vals_raw, edges_raw, ax=axH, histtype="step", label="raw", color="gray")
    hep.histplot(vals_cor, edges_raw, ax=axH, histtype="step", label="corrected", color=color)
    axH.set_ylabel("Events"); axH.legend(fontsize=11)
    axH.set_title(fr"$\mu_{{{1 if mu=='mu1' else 2}}}$ Transverse Momentum")
    centers, means = (c_mu1_x, c_mu1_y) if mu=="mu1" else (c_mu2_x, c_mu2_y)
    axC.plot(centers, means, marker="o", color="black")
    axC.axhline(1.0, ls="--", color="gray")
    axC.set_xlabel(r"$p_T^{raw}$ [GeV]"); axC.set_ylabel(r"$\langle \mathrm{corr} \rangle$")

for ax in fig.axes:
    ax.grid(True, linestyle="--", alpha=0.4)

plt.tight_layout(pad=2.0, h_pad=2.0, w_pad=2.5)

# Add watermark
fig.text(
    0.5, 0.5, "FAKE",
    color="red",
    fontsize=180,
    ha="center",
    va="center",
    alpha=0.12,
    rotation=30,
    weight="bold",
    transform=fig.transFigure,
)

plt.savefig("fake_z.png")
plt.show()
